The following is the code for simulating SMA, it assumes the preference lists of users and servers are already given


In [ ]:
# this cell basically imports all the important libraries and stuff
#importing the neccesary libraries
from math import sqrt,log2
import numpy as np
import random
import collections   
from sortedcontainers import SortedDict

In [ ]:
# basically initialises the no of users and servers
NO_OF_USERS = 100
NO_OF_BROADCAST_SERVERS = 10
BW = 10 #bandwidth 
alpha = 2.5# txf
NOISE = 0.0005


In [ ]:
# both classes defined, might need to twerk the definitions a bit
class User:
    def __init__(self,uID,loc):
        self.preference_list =[]
        self.uID = uID
        self.X = loc[0]
        self.Y = loc[1]
        self.target = None
class BS :
    def __init__(self,bID,loc):
        self.preference_list = []
        self.bID = bID
        self.X = loc[0]
        self.Y = loc[1]
        self.associatedUsers = set()  #use update to add elements to set else use add for element


In [ ]:
#this generates 110 x and y co ordinates and hence the first 100 will be taken as users
# the next 10 as routers

random.seed(1)
x_coord = random.sample(range(1,500),110)
random.seed(2)
y_coord = random.sample(range(1,500),110)

Users = [User(i,(x_coord[i],y_coord[i])) for i in range(100)]
BSs = [BS(i-99,(x_coord[i],y_coord[i])) for i in range(100,110)]

# the BS id s needed to be twerked around a bit
# BSs = [BS(i+100,(x_coord[i],y_coord[i])) for i in range(100,110)]


#print(x_coord,"\n",y_coord)

In [ ]:
# this is the distance matrix which has the distance between any random user and router
# the routers are being referred to as BS

distance_Mat =[sqrt((Users[j].X -BSs[i].X )**2 + (Users[j].Y -BSs[i].Y )**2) for i in range(0,10) for j in range (0,100)]
distance_Mat = (np.array(distance_Mat)).reshape(10,100)


In [ ]:
#User preference list 
def preflistBuilderforUsers():
    for i in range(NO_OF_USERS):
        pref_dict =SortedDict()
        pref_list =[] 
        for j in range(NO_OF_BROADCAST_SERVERS):
            a = distance_Mat[j][i]
            pref_dict[a] = j+1
        for key,value in pref_dict.items():
            pref_list.append(value)
        Users[i].preference_list = pref_list
    
      

In [ ]:
#BS preference list

"""B = BW/no of user * (1+log2(1+SINR))
SINR = rx / sum of interferences + noise """



def preflistBuilder(n:int):
    rx = alpha/(distance_Mat**2)
    interactions= np.zeros((10,100))
    for i in range(10):
        for j in range(100):
            interactions[i][j] = np.sum([rx[p][j]for p in range(10) if p!=i])

    pref_dict =collections.OrderedDict()
    pref_list =[]        
    for i in range(10):
        for j in range(100):
            a = (BW/n)*log2(1+ (rx[i][j]/(NOISE + interactions[i][j])))
            pref_dict[a]=j+1
        for key,value in pref_dict.items():
            pref_list.append(value)
        BSs[i].preference_list = pref_list
            
    

In [ ]:
n1 = NO_OF_USERS
n2 = NO_OF_BROADCAST_SERVERS
r = n1/n2
list1=[]
list2=[]
preflistBuilderforUsers()
preflistBuilder(10)

for i in range(n1):
  list1.append(Users[i].preference_list)
for i in range(n2):
  list2.append(BSs[i].preference_list)

print(list1)
print(list2)

[[5, 8, 4, 9, 1, 2, 10, 6, 7, 3], [8, 4, 2, 5, 1, 9, 7, 3, 10, 6], [4, 8, 2, 7, 5, 1, 9, 3, 10, 6], [4, 8, 2, 7, 1, 5, 9, 3, 10, 6], [3, 1, 9, 7, 2, 6, 10, 5, 4, 8], [6, 10, 9, 3, 1, 5, 2, 7, 8, 4], [6, 10, 3, 9, 1, 5, 2, 7, 8, 4], [10, 6, 9, 5, 1, 3, 2, 8, 7, 4], [5, 8, 4, 2, 1, 9, 7, 3, 10, 6], [3, 1, 9, 7, 2, 5, 10, 6, 4, 8], [5, 9, 2, 1, 8, 4, 7, 10, 3, 6], [5, 8, 2, 4, 1, 9, 7, 10, 3, 6], [2, 1, 7, 9, 8, 4, 5, 3, 10, 6], [5, 8, 4, 2, 9, 1, 7, 10, 3, 6], [3, 1, 7, 9, 2, 5, 4, 8, 10, 6], [10, 6, 9, 1, 3, 5, 2, 7, 8, 4], [5, 10, 9, 6, 1, 2, 8, 3, 4, 7], [3, 9, 1, 10, 6, 2, 5, 7, 8, 4], [5, 8, 10, 9, 1, 4, 2, 6, 7, 3], [3, 1, 7, 9, 2, 6, 10, 5, 4, 8], [7, 2, 1, 4, 9, 8, 3, 5, 10, 6], [5, 9, 1, 2, 8, 4, 10, 7, 3, 6], [3, 9, 6, 1, 10, 5, 2, 7, 8, 4], [1, 9, 3, 2, 7, 5, 8, 4, 10, 6], [2, 7, 1, 4, 8, 9, 5, 3, 10, 6], [3, 1, 7, 9, 2, 5, 4, 8, 10, 6], [5, 10, 9, 1, 6, 8, 2, 4, 7, 3], [2, 4, 8, 7, 1, 9, 5, 3, 10, 6], [5, 8, 4, 2, 9, 1, 7, 10, 3, 6], [5, 9, 10, 1, 6, 2, 3, 8, 7, 4], [3, 1, 9,

In [ ]:
# n1 is the number of users we have right now
# n2 is the number of routers we have 
# r is the ratio n1 : n2

In [ ]:
# random.sample(range(100), 10)
import random
# n1=25
# n2=5
# r=5
# list1=[]
# list2=[]

for i in range(n1):
  list1.append(random.sample(range(101,101+n2),n2))
for i in range(n2):
  list2.append(random.sample(range(1,1+n1),n1))

print(list1)
print(list2)

router_start=101
user_start=1
# rPatner=stable(list1,list2,r)
# for i in range(n2):
#   print("Routers: ")
#   print(i + router_start)
#   print("Users: ")
#   for j in rPatner[i]:
#     print(j)  


[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3,

In [ ]:
# this following function allows a router to have r users connected with it
# it also assumes the preferences of routers available are correct

In [ ]:
def stable(list1 ,list2,ratio):
  # list of patners of all routers
  
  #all users are free right now; i.e while initialising
  uNotFree = [False for i in range(n1)]
  #no of users who are still not married
  freeCount=n1

  rPatner=[]
  for i in range(n2):
    rPatner.append([])


  while (freeCount > 0):
    #this loop finds a user who is free
    u = 0
    while (u < n1):
      if (uNotFree[u]==False):
        break
      u+=1
    i = 0
    #this loop checks for each and every router in the list of user 'u'
    u+=user_start
    while i < n2 and uNotFree[u-user_start]==False:
      r = list1[u-user_start][i]
    
      if (len(rPatner[r-router_start]) < ratio):
        if (u in rPatner[r-router_start]):
          break
        else :
          rPatner[r-router_start].append(u)
          uNotFree[u-user_start]=True
          freeCount-=1
        # here you need to update the preference list of the router 
        # and hence dyanamically do it
      else :
        # find if there exists some patner whose priority is lesser than 'u'
        index_of_router=r-router_start
        index_u = list2[index_of_router].index(u)
        for x in range(ratio):
          curr = rPatner[index_of_router][x]
          index_of_curr_in_preferences = list2[index_of_router].index(curr)
          if (index_of_curr_in_preferences > index_u):
            uNotFree[curr-user_start]=False
            rPatner[index_of_router][x]=u
            uNotFree[u-user_start]=True
            break

          
         
      i+=1


  print(rPatner)          


      
  

In [ ]:
stable(list1,list2,r)


IndexError: ignored